In [2]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from env_vars import sentinel_username,sentinel_password
import glob
import pandas as pd

In [3]:
def get_api():
    
    return SentinelAPI(sentinel_username, sentinel_password, "https://scihub.copernicus.eu/apihub/")

In [4]:
api = get_api()

help(api.query)

Help on method query in module sentinelsat.sentinel:

query(area=None, date=None, raw=None, area_relation='Intersects', order_by=None, limit=None, offset=0, **keywords) method of sentinelsat.sentinel.SentinelAPI instance
    Query the OpenSearch API with the coordinates of an area, a date interval
    and any other search keywords accepted by the API.
    
    Parameters
    ----------
    area : str, optional
        The area of interest formatted as a Well-Known Text string.
    date : tuple of (str or datetime) or str, optional
        A time interval filter based on the Sensing Start Time of the products.
        Expects a tuple of (start, end), e.g. ("NOW-1DAY", "NOW").
        The timestamps can be either a Python datetime or a string in one of the
        following formats:
    
            - yyyyMMdd
            - yyyy-MM-ddThh:mm:ss.SSSZ (ISO-8601)
            - yyyy-MM-ddThh:mm:ssZ
            - NOW
            - NOW-<n>DAY(S) (or HOUR(S), MONTH(S), etc.)
            - NOW+<n

In [6]:
footprint = geojson_to_wkt(read_geojson('./data/republic_of_the_congo_boundary.geojson'))

footprint

'MULTILINESTRING((14.4291 2.1559,14.4337 2.1559,14.4401 2.1594,14.4456 2.1594,14.4908 2.1774,14.5253 2.1880,14.5290 2.1900,14.5388 2.1954,14.5459 2.1970,14.5570 2.2115,14.5674 2.2036,14.5733 2.2029,14.5809 2.2111,14.5832 2.2118,14.5866 2.2104,14.5912 2.2017,14.5939 2.2007,14.6008 2.1984,14.6067 2.1968,14.6122 2.1908,14.6163 2.1807,14.6198 2.1746,14.6306 2.1563,14.6425 2.1551,14.6478 2.1512,14.6494 2.1468,14.6457 2.1378,14.6509 2.1291,14.6560 2.1275,14.6800 2.1300,14.6859 2.1306,14.6905 2.1285,14.6989 2.1197,14.7026 2.1092,14.7149 2.1082,14.7231 2.1038,14.7268 2.1038,14.7270 2.1080,14.7223 2.1151,14.7225 2.1190,14.7241 2.1220,14.7296 2.1222,14.7554 2.1035,14.7568 2.1007,14.7554 2.0950,14.7437 2.0953,14.7423 2.0911,14.7444 2.0874,14.7499 2.0867,14.7613 2.0947,14.7721 2.0954,14.7753 2.0926,14.7773 2.0862,14.7766 2.0807,14.7702 2.0683,14.7708 2.0657,14.7750 2.0616,14.7800 2.0611,14.7942 2.0719,14.7976 2.0717,14.8104 2.0581,14.8147 2.0576,14.8276 2.0624,14.8358 2.0704,14.8367 2.0719,14.8411

In [24]:
point = 'POINT(21.04963 0.67324)'

In [11]:
def get_products(api, footprint, date_start, date_end,
                 area='Intersects', raw='1C',
                 platform='Sentinel-2', cloudcover=(0,10)):
    
    return api.query(footprint,
                     date=(date_start, date_end),
                     area_relation=area,
                     raw=raw,
                     platformname=platform,
                     cloudcoverpercentage=cloudcover)

In [25]:
products = get_products(api, point, date(2020,1,1), date(2020,2,1))

products_df = api.to_dataframe(products)

In [26]:
products_df.head()

,title,link,link_alternative,link_icon,summary,datatakesensingstart,beginposition,endposition,ingestiondate,orbitnumber,...,size,s2datatakeid,producttype,platformidentifier,orbitdirection,platformserialidentifier,processinglevel,identifier,level1cpdiidentifier,uuid
ed760c47-72cf-445c-acd4-f6c982807220,S2A_MSIL1C_20200125T085241_N0208_R107_T34NDF_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-25T08:52:41.024Z, Instrument: MS...",2020-01-25 08:52:41.024,2020-01-25 08:52:41.024,2020-01-25 08:52:41.024,2020-01-25 12:59:50.685,23985,...,703.13 MB,GS2A_20200125T085241_023985_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20200125T085241_N0208_R107_T34NDF_2...,S2A_OPER_MSI_L1C_TL_EPAE_20200125T102418_A0239...,ed760c47-72cf-445c-acd4-f6c982807220
8d0c3649-ab5e-447e-b6f1-a60a8fed91ff,S2A_MSIL1C_20200115T085311_N0208_R107_T34NEF_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-15T08:53:11.024Z, Instrument: MS...",2020-01-15 08:53:11.024,2020-01-15 08:53:11.024,2020-01-15 08:53:11.024,2020-01-15 14:48:46.404,23842,...,278.32 MB,GS2A_20200115T085311_023842_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20200115T085311_N0208_R107_T34NEF_2...,S2A_OPER_MSI_L1C_TL_EPAE_20200115T102200_A0238...,8d0c3649-ab5e-447e-b6f1-a60a8fed91ff
b89fceb9-b9ff-4fc9-9142-d68375a5c576,S2A_MSIL1C_20200115T085311_N0208_R107_T34NDF_2...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,https://scihub.copernicus.eu/apihub/odata/v1/P...,"Date: 2020-01-15T08:53:11.024Z, Instrument: MS...",2020-01-15 08:53:11.024,2020-01-15 08:53:11.024,2020-01-15 08:53:11.024,2020-01-15 14:46:32.075,23842,...,720.67 MB,GS2A_20200115T085311_023842_N02.08,S2MSI1C,2015-028A,DESCENDING,Sentinel-2A,Level-1C,S2A_MSIL1C_20200115T085311_N0208_R107_T34NDF_2...,S2A_OPER_MSI_L1C_TL_EPAE_20200115T102200_A0238...,b89fceb9-b9ff-4fc9-9142-d68375a5c576


In [27]:
products_df['tileid']

ed760c47-72cf-445c-acd4-f6c982807220    34NDF
8d0c3649-ab5e-447e-b6f1-a60a8fed91ff    34NEF
b89fceb9-b9ff-4fc9-9142-d68375a5c576    34NDF
Name: tileid, dtype: object